# Algo trading project


In [ ]:
!pip3 install alpaca-trade-api

## Initialization

In [ ]:
from alpaca.data.timeframe import TimeFrame
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical import StockHistoricalDataClient# Create stock historical data client
import pandas as pd
import os

# Set API key and secret
api_key = 'your_api_key'
api_secret = 'your_api_secret'

# Initialize trading client for live trading (using paper trading mode)
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, StopLossOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from time import sleep

trading_client = TradingClient(api_key, api_secret, paper=True)

## Define backtesting strategy

In [ ]:
# Helper function to simulate trading for backtesting
def backtest_strategy(historical_data, short_window=50, long_window=200):
    historical_data['short_ma'] = historical_data['close'].rolling(window=short_window).mean()
    historical_data['long_ma'] = historical_data['close'].rolling(window=long_window).mean()
    
    buy_price = None
    total_profit = 0
    trades = []

    for index, row in historical_data.iterrows():
        short_ma = row['short_ma']
        long_ma = row['long_ma']
        current_price = row['close']

        # Check if short MA crosses above long MA (buy signal)
        if short_ma > long_ma and buy_price is None:
            buy_price = current_price
            print(f"Simulated Buy at: {buy_price}")
        
        # Check if short MA crosses below long MA (sell signal)
        elif short_ma < long_ma and buy_price is not None:
            profit = current_price - buy_price
            total_profit += profit
            trades.append(profit)
            print(f"Simulated Sell at: {current_price}, Profit: {profit}")
            buy_price = None  

    return total_profit, trades

## Load historical data

In [ ]:
# Load historical data for backtesting
client = StockHistoricalDataClient(api_key, api_secret)
request_params = StockBarsRequest(
    symbol_or_symbols=["TSLA"],
    timeframe=TimeFrame.Day,
    start="2023-01-01 00:00:00",
    end="2023-06-01 00:00:00"
)
historical_data = client.get_stock_bars(request_params).df
print(historical_data.tail())

## Backtest the strategy

In [ ]:
total_profit, trades = backtest_strategy(historical_data)
print(f"Total Profit from Backtesting: {total_profit}")
print(f"Individual Trades: {trades}")

# Live paper trading

In [ ]:
# Define parameters
symbol = "TSLA"
short_window = 50
long_window = 200
buy_qty = 10

# Initialize performance tracking
trade_log = []
total_profit = 0
total_trades = 0

def log_trade(order, trade_type, price):
    global total_profit, total_trades
    if trade_type == 'buy':
        trade_log.append({
            'type': 'buy',
            'price': price,
            'timestamp': pd.Timestamp.now()
        })
    elif trade_type == 'sell':
        last_trade = next((t for t in reversed(trade_log) if t['type'] == 'buy'), None)
        if last_trade:
            profit = price - last_trade['price']
            total_profit += profit
            total_trades += 1
            trade_log.append({
                'type': 'sell',
                'price': price,
                'timestamp': pd.Timestamp.now(),
                'profit': profit
            })
            print(f"Trade executed: Buy at {last_trade['price']}, Sell at {price}, Profit: {profit}")

def get_moving_averages():
    request_params = StockBarsRequest(
        symbol_or_symbols=[symbol],
        timeframe=TimeFrame.Day,
        start="2023-01-01 00:00:00"
    )
    historical_data = client.get_stock_bars(request_params).df
    historical_data['short_ma'] = historical_data['close'].rolling(window=short_window).mean()
    historical_data['long_ma'] = historical_data['close'].rolling(window=long_window).mean()
    return historical_data

In [ ]:
def execute_trade():
    historical_data_live = get_moving_averages()
    last_row = historical_data_live.iloc[-1]
    previous_row = historical_data_live.iloc[-2]

    if last_row['short_ma'] > last_row['long_ma'] and previous_row['short_ma'] <= previous_row['long_ma']:
        # Place a market buy order
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=buy_qty,
            side=OrderSide.BUY,
            time_in_force=TimeInForce.GTC,
        )
        market_order = trading_client.submit_order(order_data=market_order_data)
        buy_price = float(market_order.filled_avg_price)
        print(f"Market buy order placed at {buy_price}")
        log_trade(market_order, 'buy', buy_price)

        # Set a stop-loss 5% below the buy price
        stop_loss_price = buy_price * 0.95
        stop_loss_order_data = StopLossOrderRequest(
            symbol=symbol,
            qty=buy_qty,
            side=OrderSide.SELL,
            stop_price=stop_loss_price,
            time_in_force=TimeInForce.GTC
        )
        trading_client.submit_order(order_data=stop_loss_order_data)
        print(f"Stop-loss order placed at {stop_loss_price}")

    elif last_row['short_ma'] < last_row['long_ma'] and previous_row['short_ma'] >= previous_row['long_ma']:
        # Place a market sell order (if in position)
        market_order_data = MarketOrderRequest(
            symbol=symbol,
            qty=buy_qty,
            side=OrderSide.SELL,
            time_in_force=TimeInForce.GTC,
        )
        market_order = trading_client.submit_order(order_data=market_order_data)
        sell_price = float(market_order.filled_avg_price)
        print(f"Market sell order placed at {sell_price}")
        log_trade(market_order, 'sell', sell_price)

In [ ]:
# Monitor live price updates and adjust positions
while True:
    try:
        execute_trade()
        # Print performance metrics
        print(f"Total Trades: {total_trades}, Total Profit: {total_profit}")
        sleep(60)  # Wait 1 minute before the next iteration
    except Exception as e:
        print(f"An error occurred: {e}")